# Sample run for Fisher Market

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


In [20]:
using Revise
using Random, SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf
import MathOptInterface as MOI
using Plots, LaTeXStrings, Printf, JLD2
using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
include("./setup.jl")
switch_to_pdf(; bool_use_html=true)


@load "../ml-32m.jld2" S
bool_part = true
if bool_part
    m = 2000
    n = 1000
    T = S[1:n, 1:m]
else
end
S, cols, rows = ExchangeMarket.drop_empty(T)
n, m = size(S)
# S = Matrix(S)
S = S .* 2.0
ϵₚ = 1e-6


1.0e-6

## Test different $\rho$ in a CES Economy

Run CES economy by different methods

In [21]:
method_filter(name) = name ∈ [:LogBar, :PathFol, :Tât, :PropRes]

method_filter (generic function with 1 method)

In [22]:
table_time = []
results = []
results_phi = Dict()
results_ground = Dict()
# for ρ in rrange
# ρ = -0.92
ρ = 0.6

f0 = FisherMarket(m, n; c=S, ρ=ρ, bool_unit=true, bool_unit_wealth=true, scale=0.1, bool_ensure_nz=true)
# f0 = FisherMarket(m, n; ρ=ρ, bool_unit=true, scale=30.0, sparsity=0.2)

linconstr = LinearConstr(1, n, ones(1, n), [sum(f0.w)])
ρfmt = @sprintf("%+.2f", ρ)
σfmt = @sprintf("%+.2f", f0.σ)
# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) * sum(f1.w) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
f1.p .= p₀
# use log-barrier method to compute ground truth
(name, method, kwargs) = method_kwargs[1]
kwargs = Dict(
    :tol => 1e-12, :maxiter => 20,
    :optimizer => CESAnalytic,
    :option_mu => :pred_corr,
    # :option_mu => :normal,
    #
    # :option_step => :affinesc,
    # :option_step => :homotopy,
    :option_step => :logbar,
    # :linsys => :DRq,
    :linsys => :krylov,
    # :linsys => :direct,
)
alg = method(
    n, m, p₀;
    linconstr=linconstr,
    kwargs...
)
traj = opt!(
    alg, f1;
    loginterval=1,
    maxiter=100,
    keep_traj=true,
    bool_init_phase=false,
)
pₛ = copy(alg.p)
results_phi[ρ] = pₛ
results_ground[ρ] = (alg, traj, f1);


FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0031189918518066406 seconds
FisherMarket initialized in 0.00321197509765625 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.002148866653442383 seconds
FisherMarket initialized in 0.0022020339965820312 seconds
--------------------------------------------------------------------------------------------
                   ExchangeMarket.jl: A Julia Package for Exchange Market                   
                                    © Chuwen Zhang (2024)                                    
--------------------------------------------------------------------------------------------
 subproblem solver alias       := CESAnalytic
 subproblem solver style       := analytic
 lin-system solver alias       := krylov
 option for gradient           := dual
 option for step               := logbar
 option for μ                  := pred_corr
------------------------------------------

In [5]:
s = [1]
s[]

1

In [6]:
using LinearAlgebra, LinearOperators, Krylov

In [ ]:
alg.Hk.niter

In [ ]:
f1.σ

In [ ]:
@time ExchangeMarket.__compute_exact_hess_optimized!(alg, f1)
H = diagm(alg.p) * alg.H * diagm(alg.p)

# fisher = f1
# b = alg.p .* fisher.x
# pxbar = sum(b; dims=2)[:]
# γ = 1 ./ fisher.w' .* b
# u = fisher.w .* fisher.σ
# H₁ = diagm(pxbar .* (fisher.σ + 1)) - γ * diagm(u) * γ'
# H₁ - H

## Baseline

In [ ]:
ExchangeMarket.__update_php_hessop!(alg, f1)
d, stats = cg(alg.Hk.php_hessop, alg.p .* alg.∇; verbose=3, timemax=10.0, history=true)

In [ ]:
ExchangeMarket.__update_php_hessop!(alg, f1)
d, stats = cg(H, alg.p .* alg.∇; verbose=3, timemax=10.0)

In [14]:
using RandomizedPreconditioners

In [ ]:
buff = zeros(n)
v = rand(n)
ExchangeMarket.__update_php_hessop!(alg, f1)
mul!(buff, alg.Hk.php_hessop, v)
@info "double check Hessian is correct" norm(buff - (H + alg.μ * I) * v)

In [ ]:
Â = NystromSketch(H, 100; n=f1.n)
P = NystromPreconditionerInverse(Â, 1e-3)

In [ ]:
# d, stats = cg(alg.H, alg.∇ .- alg.μ ./ alg.p; verbose=3, timemax=10.0)
d, stats = cg(H, alg.p .* alg.∇ .- alg.μ; M=P, verbose=3, timemax=5.0)

## Diagonal preconditioner

In [ ]:
# τ = [1 / norm(H[:, i], 1) for i = 1:n]  # diagonal preconditioner
τ = 1 ./ sum(H; dims=2)[:]  # diagonal preconditioner
P⁻¹ = diagm(τ)
d, stats = cg(H, alg.p .* alg.∇ .- alg.μ; M=P⁻¹, rtol=1e-15, verbose=3, timemax=5.0)

In [ ]:
ExchangeMarket.__update_php_hessop!(alg, f1)
a = rand(n)
d, stats = cg(alg.Hk.php_hessop, alg.p .* alg.∇, a; verbose=3, timemax=10.0, history=true)
τ = 1 ./ sum(H; dims=2)[:]  # diagonal preconditioner
P⁺ = diagm(sqrt.(τ))
d1, stats1 = cg(H, alg.p .* alg.∇ .- alg.μ, a; M=P⁻¹, rtol=1e-15, verbose=3, timemax=5.0, history=true)

In [ ]:
fig = generate_empty(; shape=:wide)
plot!(
    fig,
    ylabel=L"$\|\mathbf{H}(\mathbf{p}) + \mu \mathbf{I})\mathbf{d} - \mathbf{r}\|$",
    title=L"$\rho := %$ρfmt~(\sigma := %$σfmt)$",
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    yticks=10.0 .^ (-16:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    xscale=:identity,
    size=(600, 400)
)
plot!(
    fig,
    xticks=[10, 50, 100, 200, 500]
)

plot!(fig, stats.residuals[1:4:end], label=L"\textrm{No precond.}", linewidth=2, linestyle=:dash, markershape=:circle)
plot!(fig, stats1.residuals, label=L"\textrm{Diagonal precond.}", linewidth=2, linestyle=:dash, markershape=:circle)


In [ ]:
savefig(fig, "/tmp/fisher_linsys_precond.pdf")

In [ ]:
eigen(P⁺ * H * P⁺)

In [ ]:
1.0000000000000009 / 0.09999999999999895

In [ ]:
d1, stats1 = cg(H, linconstr.A', a; M=P⁻¹, rtol=1e-15, verbose=3, timemax=5.0, history=true)